In [1]:
!pip install mord
!pip install xgboost
!pip install shap

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import statsmodels.api as sm
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from tpot import TPOTClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from tpot import TPOTClassifier
from sklearn.tree import DecisionTreeClassifier
from mord import OrdinalRidge
import xgboost as xgb
from keras.layers import Input
from keras_tuner import RandomSearch
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import shap
from tabulate import tabulate
import requests
from io import StringIO

In [3]:
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/train_data.csv?token=GHSAT0AAAAAACC4ZCNLK5WDAXMHGAA2JI24ZGGSK4A"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'train' variable
    train = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'train' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/val_data.csv?token=GHSAT0AAAAAACC4ZCNK6U7OS5YW72PK3GBSZGGSH2A"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'valid' variable
    valid = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'valid' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/test_data.csv?token=GHSAT0AAAAAACC4ZCNL45CUOECJPHCIM43GZGGSIGQ"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'train' variable
    test = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'test' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")

In [4]:
# Convert variables to numeric in the train dataset
variables_to_convert = train.columns.drop('Date')
train[variables_to_convert] = train[variables_to_convert].apply(pd.to_numeric, errors='coerce')

# Convert variables to numeric in the valid dataset
valid[variables_to_convert] = valid[variables_to_convert].apply(pd.to_numeric, errors='coerce')

In [5]:
train['Direction'] = np.select([
    train['Difference'] > 0,
    train['Difference'] < 0,
    train['Difference'] == 0
], [
    1,  # Increase
    -1,  # Decrease
    0   # Hold
], default=-1)

valid['Direction'] = np.select([
    valid['Difference'] > 0,
    valid['Difference'] < 0,
    valid['Difference'] == 0
], [
    1,  # Increase
    -1,  # Decrease
    0   # Hold
], default=-1)

In [6]:
print(train.columns)

Index(['Increase', 'Decrease', 'Level', 'CPI', 'Unemployment Rate',
       'Retail Sales', 'Durable Goods Orders', 'Short-Term Treasury Bond Rate',
       'Long-Term Treasury Bond Rate', 'Bank Reserves', 'LEI', 'CEI', 'LAG',
       'Treasury Deposits', 'Average Hourly Earnings', 'Net Sentiment Score',
       'Positive Frequency', 'Negative Frequency', 'Word Count',
       'Standardized Sentiment Score', 'Short-Term Treasury Diff',
       'LEI_RollingMean', 'CEI_RollingMean', 'LAG_RollingMean',
       'Retail Sales_RollingMean', 'Durable Goods Orders_RollingMean',
       'almost', 'anticipated', 'anticipating', 'apparent', 'apparently',
       'appeared', 'appearing', 'assumed', 'assumption', 'attain',
       'attractive', 'attractiveness', 'believed', 'better', 'bolstered',
       'bolstering', 'cautious', 'cautiously', 'could', 'depend', 'easier',
       'encouragement', 'favorable', 'gaining', 'good', 'improved',
       'improvement', 'leading', 'might', 'nearly', 'opportunity',
    

###Directional Prediction

###Ordinal Ridge Regression

In [7]:
# Prepare the data for the model
X_train = train.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_train = train['Direction']  # Use the 'Direction' variable as the target

X_valid = valid.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_valid = valid['Direction']  # Use the 'Direction' variable as the target

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Initialize and train the ordinal logistic regression model
ordinal_model = OrdinalRidge()
ordinal_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_valid_pred = ordinal_model.predict(X_valid_scaled)

# Make predictions on the training set
y_train_pred = ordinal_model.predict(X_train_scaled)

In [8]:
# Calculate Mean Squared Error (MSE) for training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error (MSE):", mse_train)

# Calculate Mean Squared Error (MSE) for validation set
mse_valid = mean_squared_error(y_valid, y_valid_pred)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training set
rmse_train = np.sqrt(mse_train)
print("Training Root Mean Squared Error (RMSE):", rmse_train)

# Calculate Root Mean Squared Error (RMSE) for validation set
rmse_valid = np.sqrt(mse_valid)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training set
r2_train = r2_score(y_train, y_train_pred)
print("Training R^2:", r2_train)

# Calculate R-squared (R^2) for validation set
r2_valid = r2_score(y_valid, y_valid_pred)
print("Validation R^2:", r2_valid)

# Calculate adjusted R-squared for training set
n_train = X_train.shape[0]  # Number of samples in training set
p_train = X_train.shape[1]  # Number of features in training set
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
print("Training Adjusted R^2:", adj_r2_train)

# Calculate adjusted R-squared for validation set
n_valid = X_valid.shape[0]  # Number of samples in validation set
p_valid = X_valid.shape[1]  # Number of features in validation set
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Validation Adjusted R^2:", adj_r2_valid)

Training Mean Squared Error (MSE): 0.17801047120418848
Validation Mean Squared Error (MSE): 0.5
Training Root Mean Squared Error (RMSE): 0.4219128715791786
Validation Root Mean Squared Error (RMSE): 0.7071067811865476
Training R^2: 0.39903757171941523
Validation R^2: -0.22033898305084754
Training Adjusted R^2: -0.18940480597199083
Validation Adjusted R^2: 1.3953210790164716


In [9]:
# Initialize and train the ordinal logistic regression model
ordinal_model = OrdinalRidge()
ordinal_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_valid_pred = ordinal_model.predict(X_valid_scaled)

# Make predictions on the training set
y_train_pred = ordinal_model.predict(X_train_scaled)

# Convert the predicted values to ordinal categories
y_train_pred_category = np.round(y_train_pred).astype(int)
y_valid_pred_category = np.round(y_valid_pred).astype(int)

# Calculate accuracy for training and validation sets
accuracy_train = accuracy_score(y_train, y_train_pred_category)
accuracy_valid = accuracy_score(y_valid, y_valid_pred_category)

print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Training Accuracy: 0.8219895287958116
Validation Accuracy: 0.5


In [10]:
# Access the coefficients and intercepts
coefficients = ordinal_model.coef_
intercepts = ordinal_model.intercept_

# Display coefficients and intercepts
print("Coefficients:")
print(coefficients)

print("Intercepts:")
print(intercepts)

Coefficients:
[ 0.02402709 -0.08662731 -0.15515958  0.14026437 -0.02640148 -0.03135705
 -0.15349031 -0.01867492 -0.16180556 -0.02017475 -0.01504331 -0.05897942
 -0.15792497  0.01118825  0.00359184 -0.00099917 -0.08670814  0.01118825
  0.31060554  0.03835019  0.02974458  0.0336391   0.30179174 -0.2238576
 -0.02915485 -0.01829946  0.12587379  0.05849727 -0.02207385  0.09471137
 -0.1053766  -0.05668885 -0.05741903  0.02104695  0.0189186  -0.01731716
 -0.01223244  0.11684601 -0.0643666  -0.04351069 -0.06576789 -0.00441159
  0.20326871 -0.1015785   0.08127378 -0.07471455  0.00067962 -0.04500762
  0.26533477 -0.13220994  0.10739167 -0.00710719 -0.17400346 -0.09744961
 -0.06866535  0.06749356  0.00143774 -0.07296422 -0.02882997 -0.03363
 -0.07990339 -0.06529572  0.02111175 -0.1220375  -0.05129406  0.02012124
 -0.0021426  -0.08774782 -0.03045944  0.04898673 -0.03176268 -0.01020718
  0.00399995 -0.05599647 -0.10103282 -0.13480984 -0.15984486 -0.03711265
  0.05295604 -0.06143211 -0.07283566 -0.0

###XGBoost Model

In [11]:
# Prepare the data for the model
X_train = train.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_train = train['Direction']  # Use the 'Direction' variable as the target

X_valid = valid.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_valid = valid['Direction']  # Use the 'Direction' variable as the target

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)

# Make predictions on the validation set
y_valid_pred = xgb_model.predict(X_valid)

In [12]:
# Calculate Mean Squared Error (MSE) for training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error (MSE):", mse_train)

# Calculate Mean Squared Error (MSE) for validation set
mse_valid = mean_squared_error(y_valid, y_valid_pred)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training set
rmse_train = np.sqrt(mse_train)
print("Training Root Mean Squared Error (RMSE):", rmse_train)

# Calculate Root Mean Squared Error (RMSE) for validation set
rmse_valid = np.sqrt(mse_valid)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training set
r2_train = r2_score(y_train, y_train_pred)
print("Training R^2:", r2_train)

# Calculate R-squared (R^2) for validation set
r2_valid = r2_score(y_valid, y_valid_pred)
print("Validation R^2:", r2_valid)

# Calculate adjusted R-squared for training set
n_train = X_train.shape[0]  # Number of samples in training set
p_train = X_train.shape[1]  # Number of features in training set
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
print("Training Adjusted R^2:", adj_r2_train)

# Calculate adjusted R-squared for validation set
n_valid = X_valid.shape[0]  # Number of samples in validation set
p_valid = X_valid.shape[1]  # Number of features in validation set
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Validation Adjusted R^2:", adj_r2_valid)

Training Mean Squared Error (MSE): 3.2998337238190296e-07
Validation Mean Squared Error (MSE): 0.23903423250877073
Training Root Mean Squared Error (RMSE): 0.0005744417919875807
Validation Root Mean Squared Error (RMSE): 0.48891127263417716
Training R^2: 0.9999988859778449
Validation R^2: 0.41659441557181376
Training Adjusted R^2: 0.9999977951644847
Validation Adjusted R^2: 1.1889905414344828


In [13]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Extract feature importance
feature_importance = xgb_model.feature_importances_

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = {feature_name: importance_score for feature_name, importance_score in zip(X_train.columns, feature_importance)}

# Sort the feature importance dictionary in descending order based on importance scores
sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True))

# Print the feature importance scores in descending order
print("Feature Importance:")
for feature_name, importance_score in sorted_feature_importance.items():
    print(f"{feature_name}: {importance_score:.4f}")

Feature Importance:
Short-Term Treasury Diff: 0.2838
Word Count: 0.1056
Short-Term Treasury Bond Rate: 0.1025
depend: 0.0745
better: 0.0509
CEI_RollingMean: 0.0351
assumption: 0.0346
uncertainty: 0.0330
LEI: 0.0317
Treasury Deposits: 0.0280
encouragement: 0.0276
CEI: 0.0253
Positive Frequency: 0.0219
Level: 0.0216
Bank Reserves: 0.0198
LAG_RollingMean: 0.0191
Unemployment Rate: 0.0174
anticipating: 0.0100
Average Hourly Earnings: 0.0077
Net Sentiment Score: 0.0075
Negative Frequency: 0.0066
anticipated: 0.0065
risk: 0.0057
possibility: 0.0043
progress: 0.0037
appeared: 0.0022
suggesting: 0.0018
Durable Goods Orders: 0.0013
CPI: 0.0013
could: 0.0012
favorable: 0.0008
apparent: 0.0007
Retail Sales: 0.0006
Durable Goods Orders_RollingMean: 0.0006
Long-Term Treasury Bond Rate: 0.0006
unclear: 0.0006
stabilized: 0.0005
believed: 0.0005
LEI_RollingMean: 0.0005
improvement: 0.0004
stronger: 0.0004
bolstered: 0.0003
good: 0.0002
volatile: 0.0002
uncertain: 0.0001
possibly: 0.0001
unexpected: 0

In [14]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)

# Make predictions on the validation set
y_valid_pred = xgb_model.predict(X_valid)

# Convert the predicted values to ordinal categories
y_train_pred_category = np.round(y_train_pred).astype(int)
y_valid_pred_category = np.round(y_valid_pred).astype(int)

# Calculate accuracy for training and validation sets
accuracy_train = accuracy_score(y_train, y_train_pred_category)
accuracy_valid = accuracy_score(y_valid, y_valid_pred_category)

print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Training Accuracy: 1.0
Validation Accuracy: 0.75


###RNN Model

In [15]:
# Preprocess the data
X_train = train.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference', 'Date'])
y_train = train['Direction']

X_valid = valid.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference', 'Date'])
y_valid = valid['Direction']

# Encode the ordinal 'Direction' variable to numerical values (e.g., -1, 0, 1)
y_train_encoded = pd.Series(np.where(y_train == 'Decrease', -1, np.where(y_train == 'Hold', 0, 1)))
y_valid_encoded = pd.Series(np.where(y_valid == 'Decrease', -1, np.where(y_valid == 'Hold', 0, 1)))

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Prepare the input sequences for the RNN model (assuming sequence_length is defined)
def prepare_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i+sequence_length-1])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Define the length of the input sequence
X_train_seq, y_train_seq = prepare_sequences(y_train_encoded, sequence_length)
X_valid_seq, y_valid_seq = prepare_sequences(y_valid_encoded, sequence_length)

# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the model on the validation set
y_train_pred = model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Epoch 1/10
6/6 [==============================] - 6s 245ms/step - loss: 0.9943 - accuracy: 0.6484 - val_loss: 0.7901 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 22ms/step - loss: 0.6399 - accuracy: 1.0000 - val_loss: 0.3995 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 20ms/step - loss: 0.2650 - accuracy: 1.0000 - val_loss: 0.1087 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 21ms/step - loss: 0.0663 - accuracy: 1.0000 - val_loss: 0.0274 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 25ms/step - loss: 0.0186 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 25ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 24ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 8/10
6/6 [==========

In [16]:
# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Define the hyperparameter search space
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    executions_per_trial=1,
    directory='my_dir',  # Directory to store the results
    project_name='my_project'
)

# Search for the best hyperparameters
tuner.search(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Print the summary of the best model
best_model.summary()

# Train the best model
best_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the best model on the validation set
y_train_pred = best_model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = best_model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("Best Hyperparameters:")
print(best_hyperparameters.values)

INFO:tensorflow:Reloading Tuner from my_dir\my_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 198,531
Trainable params: 198,531
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
6/6 [==============================] - 5s 204ms/step - loss: 0.9635 - accuracy: 1.0000 - val_loss: 0.9212 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 24ms/step - loss

In [17]:
# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(Input(shape=(sequence_length, 1)))  # Add an input layer with the desired input shape
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Show the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 64)            16896     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 29,411
Trainable params: 29,411
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Convert 'Date' column to datetime type
train['Date'] = pd.to_datetime(train['Date'])
valid['Date'] = pd.to_datetime(valid['Date'])

# Define a reference date (you can choose any date you prefer)
reference_date = pd.to_datetime('1990-01-01')

# Extract numerical representation of 'Date' column
train['Date'] = (train['Date'] - reference_date).dt.days
valid['Date'] = (valid['Date'] - reference_date).dt.days

# Preprocess the data
X_train = train.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference'])
y_train = train['Direction']

X_valid = valid.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference'])
y_valid = valid['Direction']

# Encode the ordinal 'Direction' variable to numerical values (e.g., -1, 0, 1)
y_train_encoded = pd.Series(np.where(y_train == 'Decrease', -1, np.where(y_train == 'Hold', 0, 1)))
y_valid_encoded = pd.Series(np.where(y_valid == 'Decrease', -1, np.where(y_valid == 'Hold', 0, 1)))

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Prepare the input sequences for the RNN model (assuming sequence_length is defined)
def prepare_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i+sequence_length-1])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Define the length of the input sequence
X_train_seq, y_train_seq = prepare_sequences(y_train_encoded, sequence_length)
X_valid_seq, y_valid_seq = prepare_sequences(y_valid_encoded, sequence_length)

# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the model on the validation set
y_train_pred = model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Epoch 1/10
6/6 [==============================] - 5s 215ms/step - loss: 0.9420 - accuracy: 0.8242 - val_loss: 0.7131 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 19ms/step - loss: 0.5649 - accuracy: 1.0000 - val_loss: 0.3379 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 16ms/step - loss: 0.2149 - accuracy: 1.0000 - val_loss: 0.0727 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 21ms/step - loss: 0.0380 - accuracy: 1.0000 - val_loss: 0.0101 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 19ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 25ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 21ms/step - loss: 9.4774e-04 - accuracy: 1.0000 - val_loss: 7.2798e-04 - val_accuracy: 1.0000
Epoch 8/10
6/6 [==